In [1]:
%matplotlib inline 
import matplotlib.pyplot as plt
import numpy as np
import sounddevice as sd
from scipy.io import wavfile 
import os

In [17]:
## Set parameters
filename = "test2.txt"
# Frequency encodings in Hz
low_freq = 1000
high_freq = 5103 # Chose something that's coprime with low_freq

fs = 44100 # Usually 44100 or 48000 fps 
t = .01
max_amp = 32767 # Because of signed 16 bit integers


In [30]:
from string import ascii_lowercase, ascii_uppercase
def letter_count(file):
    with open(file) as f:
        data = f.read()
        total_count = len(data)
        text = data.strip()
        freq_dic = {}
        char_set = {c for c in data}
#         for x in ascii_lowercase + ascii_uppercase + ' ':
        for x in char_set:
            freq_dic[x] = text.count(x) / total_count
    return data, freq_dic

In [11]:
from heapq import heappush, heappop, heapify
from collections import defaultdict

def HuffEncode(freq_dict):
    """Return a dictionary (symbols2huff) which maps keys from the input dictionary freq_dict
       to bitstrings using a Huffman code based on the frequencies of each key"""
    freqToLetters = []
    huffMapSoFar = dict()
    for key in freq_dict:
        heappush(freqToLetters, ((freq_dict[key]), [key]))
        huffMapSoFar[key]= ""
    # Your Beautiful Code Here #
    while len(freqToLetters) > 1:
        smallest = heappop(freqToLetters)
        secsmallest = heappop(freqToLetters)
        combined = (smallest[0] + secsmallest[0], smallest[1] + secsmallest[1])
        for symbol in smallest[1]:
            huffMapSoFar[symbol] = "0" + huffMapSoFar[symbol]
        for symbol in secsmallest[1]:
            huffMapSoFar[symbol] = "1" + huffMapSoFar[symbol]
        heappush(freqToLetters, combined)
    return huffMapSoFar

## Make sure to remove Gold code before doing this
def HuffDecode(bit_string, symbols2huff): 
    contents = ""
    

In [27]:
contents, freq_dict = letter_count(filename)
print(freq_dict)

{'g': 0.014502307185234015, '3': 0.0006591957811470006, '?': 0.0013183915622940012, 'v': 0.01120632827949901, "'": 0.005932762030323006, '"': 0.0013183915622940012, 'T': 0.0013183915622940012, 'I': 0.01120632827949901, 'd': 0.02966381015161503, '.': 0.00922874093605801, 'l': 0.03098220171390903, 'a': 0.05471324983520105, 'u': 0.04021094264996704, 'h': 0.03295978905735003, 'N': 0.0013183915622940012, 'y': 0.02702702702702703, 'c': 0.023071852340145024, 'Q': 0.0006591957811470006, 'w': 0.017798286090969017, 'U': 0.001977587343441002, 'b': 0.01120632827949901, 'f': 0.016479894528675015, 'B': 0.0006591957811470006, 'M': 0.0006591957811470006, 'm': 0.017139090309822018, 'E': 0.0006591957811470006, 'A': 0.001977587343441002, '\n': 0.0, 'W': 0.0006591957811470006, ',': 0.01120632827949901, '-': 0.0006591957811470006, 's': 0.03361898483849703, 'n': 0.05669083717864205, 'p': 0.01054713249835201, 'x': 0.0013183915622940012, 'Y': 0.0026367831245880024, 't': 0.07053394858272907, 'S': 0.00263678312

In [28]:
huff = HuffEncode(freq_dict)
print(huff)

{'g': '100100', '3': '111111011111', '?': '1100110000', 'v': '1111011', "'": '11111100', '"': '1100100110', 'T': '1100110101', 'I': '1111001', 'd': '01111', '.': '1100101', 'l': '10011', 'a': '0101', 'u': '11100', 'h': '10100', 'N': '1100110011', 'y': '01110', 'c': '111110', 'Q': '11001101001', 'w': '110001', 'U': '110010001', 'b': '1111010', 'f': '100101', 'B': '11001100010', 'M': '11001100101', 'm': '110000', 'E': '11001100100', 'A': '110010000', '\n': '1111110111100', 'W': '11111101110', ',': '1111000', '-': '1111110111101', 's': '10101', 'n': '0110', 'p': '1100111', 'x': '1111110110', 'Y': '111111010', 't': '1011', 'S': '110011011', 'i': '0100', 'r': '11101', ' ': '00', 'o': '1000', 'P': '11001101000', 'j': '110010010', 'C': '11001100011', 'k': '1111111', 'e': '1101', '0': '1100100111'}


In [29]:
huff[',']

'1111000'

In [32]:
#Import text and store as a string

# contents = "" 
# with open(filename, "r") as f: 
#     contents = f.read() 
print("Contents as string: ", contents)

### Convert contents to binary string bin_contents 
# bin_contents = "".join(format(ord(c), '08b') for c in contents) #08b means 0 pad and 8 length
bin_contents = ""
for c in contents: 
    bin_contents += huff[c]
print("Bit representation: ", bin_contents)

##Creating the "Sandwiche" Signal##

Flag =  '11100111'*10
bin_contents = Flag+bin_contents+Flag

# Generate wave 
wave = [] 
for b in bin_contents:
    if b == '0':
        freq = low_freq 
    elif b == '1':
        freq = high_freq 
    else:
        print(b)    
    waveform = np.sin(2 * np.pi * freq * np.arange(t * fs) / fs) 
    waveform_ints = np.int16(max_amp * waveform)
    wave = np.append(wave, waveform_ints)
print("Wave shape: ", np.shape(wave))


Contents as string:  What the fuck did you just fucking say about me, you little bitch? I'll have you know I graduated top of my class in the Navy Seals, and I've been involved in numerous secret raids on Al-Quaeda, and I have over 300 confirmed kills. I am trained in gorilla warfare and I'm the top sniper in the entire US armed forces. You are nothing to me but just another target. I will wipe you the fuck out with precision the likes of which has never been seen before on this Earth, mark my fucking words. You think you can get away with saying that shit to me over the Internet? Think again, fucker. As we speak I am contacting my secret network of spies across the USA and your IP is being traced right now so you better prepare for the storm, maggot. The storm that wipes out the pathetic little thing you call your life. You're fucking dead, kid. I can be anywhere, anytime, and I can kill you in over seven hundred ways, and that's just with my bare hands. Not only am I extensively trai

Wave shape:  (3045987,)
